# Suduko Solver

Backtracking ensures correctness by enumerating all possibilities. It is wasteful
to find all permutations, then analyze for a correct answer. Prunning ensures only 
correct permutations can be generated by ensuring that a partial solution can be 
extended into a full solution. For this program, the PossibleValues function only
generates candidates for a cell that are correct. When there are no valid moves for 
a cell, the program will backtrack and attempt another possible value.

I implemented two search heuristics: Lowest Index First and Most Constrained First.

Most Constrained First performed 5.5 times faster on the easy input.

In [131]:
from IPython.display import clear_output
import time
DIMENSION = 9
MOVE_COUNT = 0
UNMOVE_COUNT = 0
class Puzzle:
    
    def __init__(self, givenCells):
        """Puzzle board class
        
        Args:
            givenCells: List[(Int,Int,Int)] -> x,y,value
        """
        self.m = [[0]*DIMENSION for _ in range(DIMENSION)]
        for y,x,val in givenCells:
            if 0 <= x < DIMENSION and 0 <= y < DIMENSION and 0 < val <= DIMENSION: 
                self.m[y][x] = val
            else:
                raise ValueError(f"Invalid value:\n\tx: {x}, y: {y}, val: {val}")
        self.free_count = DIMENSION ** 2 - len(givenCells)
        self.move = [((0,0), 0)]*(self.free_count+1)

    def __str__(self):
        string=''
        for y in range(DIMENSION):
            if not y % 3:
                string+='-'*31 + '\n' # 31 = 3*DIMENSION + 4
            for x in range(DIMENSION):
                if not x % 3:
                    string+='|'
                if self.m[y][x]:
                    string+=' '+str(self.m[y][x])+' '
                else:
                    string+='   '
            string+='|\n'
        string+='-'*31
        return string
    

In [122]:
board_easy = [
    (0,0,3),
    (0,2,5),
    (0,3,4),
    (0,4,9),
    (0,5,6),
    (1,2,8),
    (1,4,1),
    (1,6,4),
    (1,8,6),
    (2,1,6),
    (2,2,1),
    (2,5,7),
    (2,7,9),
    (3,0,5),
    (3,3,1),
    (3,5,4),
    (3,7,6),
    (4,0,6),
    (4,1,3),
    (4,4,7),
    (4,8,8),
    (5,0,1),
    (5,2,2),
    (5,6,9),
    (5,7,4),
    (5,8,7),
    (6,1,5),
    (6,4,4),
    (6,7,2),
    (7,0,7),
    (7,1,4),
    (7,5,2),
    (7,6,5),
    (8,1,1),
    (8,2,9),
    (8,3,5),
    (8,8,4)
]


board_hard = [
    (0,7,1),
    (0,8,2),
    (1,4,3),
    (1,5,5),
    (2,3,6),
    (2,7,7),
    (3,0,7),
    (3,6,3),
    (4,3,4),
    (4,6,8),
    (5,0,1),
    (6,3,1),
    (6,4,2),
    (7,1,8),
    (7,7,4),
    (8,1,5),
    (8,6,6),
]

puzzle_0 = Puzzle(board_easy)
print(puzzle_0)

-------------------------------
| 3     5 | 4  9  6 |         |
|       8 |    1    | 4     6 |
|    6  1 |       7 |    9    |
-------------------------------
| 5       | 1     4 |    6    |
| 6  3    |    7    |       8 |
| 1     2 |         | 9  4  7 |
-------------------------------
|    5    |    4    |    2    |
| 7  4    |       2 | 5       |
|    1  9 | 5       |       4 |
-------------------------------


In [258]:
def NextSquare(board):
    # Lowest index first
    for i in range(len(board.m)):
        for j in range(len(board.m[0])):
            if not board.m[i][j]:
                return j,i


In [256]:
def NextSquare(board):
    # Least constrained first
    mostConstrained = (-1,-1)
    mostConstraints = -1
    for i in range(len(board.m)):
        for j in range(len(board.m[0])):
            if not board.m[i][j]:
                if len(PossibleValues((j,i), board)) > mostConstraints:
                    mostConstrained = (j,i)
    return mostConstrained
                

In [193]:
def PossibleValues(square, board):
    possible_values = list(range(1,10))

    # row check
    for c in board.m[square[1]]:
        if c and c in possible_values:
            possible_values.remove(c)
    # column check
    for i in range(len(board.m)):
        val = board.m[i][square[0]]
        if val and val in possible_values:
            possible_values.remove(val)
    
    # sector check
    x_sector = square[0] // 3
    y_sector = square[1] // 3

    for row in board.m[3*y_sector:3*(y_sector+1)]:
        for val in row[3*x_sector:3*(x_sector+1)]:
            if val and val in possible_values:
                possible_values.remove(val)
    return possible_values



In [195]:
puzzle_0 = Puzzle(board_easy)
print(puzzle_0)
print(PossibleValues((8,0),puzzle_0))
print(PossibleValues((0,8), puzzle_0))
print(PossibleValues((2,6), puzzle_0))

-------------------------------
| 3     5 | 4  9  6 |         |
|       8 |    1    | 4     6 |
|    6  1 |       7 |    9    |
-------------------------------
| 5       | 1     4 |    6    |
| 6  3    |    7    |       8 |
| 1     2 |         | 9  4  7 |
-------------------------------
|    5    |    4    |    2    |
| 7  4    |       2 | 5       |
|    1  9 | 5       |       4 |
-------------------------------
[1, 2]
[2, 8]
[3, 6]


In [148]:
def ConstructCandidates(k, board):
    pos = NextSquare(board)
    if not pos:
        return ((-1,-1), [])
    return pos, PossibleValues(pos, board)


In [238]:
def MakeMove(k, pos, val, board):
    if board.free_count == 0:
        return
    global MOVES_MADE
    MOVES_MADE+=1
    clear_output(wait=True)
    print(f"Made Move:\n\tpos: {pos}, val: {val}\n\tFree Count: {board.free_count}")
    board.move[k] = (pos, val)
    board.m[pos[1]][pos[0]] = val
    board.free_count -= 1
    print(board)
    time.sleep(0.01)


In [239]:
def UnmakeMove(k, pos, val, board):
    if board.free_count == 0:
        return
    global MOVES_UNMADE
    MOVES_UNMADE += 1
    clear_output(wait=True)
    print(f"Unmade Move:\n\tpos: {pos}, val: {val}\n\tFree Count: {board.free_count}")
    board.move[k] = ((0,0), 0)
    board.m[pos[1]][pos[0]] = 0
    board.free_count += 1
    print(board)
    time.sleep(0.01)

In [254]:
MOVES_MADE = 0
MOVES_UNMADE = 0
def Backtrack(k, board):
    if not board.free_count:
        return
    else:
        k+=1
        pos, candidates = ConstructCandidates(k, board)
        if pos == (-1,-1): return
        for c in candidates:
            MakeMove(k, pos, c, board)
            Backtrack(k, board)
            UnmakeMove(k, pos, c, board)



In [259]:
puzzle_0 = Puzzle(board_easy)
Backtrack(0, puzzle_0)
print(f"Moves made: {MOVES_MADE}")
print(f"Number of backtracks: {MOVES_UNMADE}")
MOVES_MADE=0
MOVES_UNMADE=0

Made Move:
	pos: (7, 8), val: 3
	Free Count: 1
-------------------------------
| 3  7  5 | 4  9  6 | 8  1  2 |
| 9  2  8 | 3  1  5 | 4  7  6 |
| 4  6  1 | 8  2  7 | 3  9  5 |
-------------------------------
| 5  9  7 | 1  8  4 | 2  6  3 |
| 6  3  4 | 2  7  9 | 1  5  8 |
| 1  8  2 | 6  5  3 | 9  4  7 |
-------------------------------
| 8  5  3 | 7  4  1 | 6  2  9 |
| 7  4  6 | 9  3  2 | 5  8  1 |
| 2  1  9 | 5  6  8 | 7  3  4 |
-------------------------------
Moves made: 383
Number of backtracks: 339


In [261]:
speed_up = 383 / 69
print(speed_up)

5.550724637681159
